<a href="https://colab.research.google.com/github/BluEyeNick/Simple-MusicGen-Colab/blob/main/Simple_MusicGen_w_AutoSave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Music Generator w AutoSave**

**What?** *Generate Music with AI from a text prompt*

**Why?** *Experimenting with and learning about cutting edge AI Music generation*

**How?** *4 easy steps... no coding knowledge required!*

*   Press Connect at the top right of the screen. *(If this doesnt connect you will have to try later or buy credits from Google)*

*   Set the Settings *(see the next block)* ⬇️

*   Press the play button on the left hand side under "Settings"

*   Upon running the first time you will need to connect your google drive for auto-save. ~*(If you want a version without auto-save click here-wip)*~




In [ ]:
#@title Settings
#@markdown **Text Input:** Type in what you want to hear
prompt = "" #@param {type:"string"}
#@markdown How many times to generate the prompt?
batch_size = 1 #@param {type:"slider",min:1.0,max:99.0,step:1}
#@markdown **Duration** in seconds
seconds = 30 # @param {type:"slider", min:8, max:30, step:1}

#Insert default prompt if there was none entered
if prompt == "":
    prompt = "80s dance music"

#Check for imports
try:
    already_imported
except NameError:
    already_imported = None
else:
    print("Already Installed. Skipping Install and Imports.")

# Install and import neccessary libraries if needed
if already_imported is None:
  from google.colab import drive
  drive.mount('/content/gdrive')

  !pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft

  import os
  os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
  import numpy as np
  import torch
  import soundfile as sf

  from scipy.io.wavfile import write
  from audiocraft.models import musicgen
  from audiocraft.utils.notebook import display_audio

#@markdown **Select Model**

#@markdown *Recommend Large*

  selected_model = "large" #@param ["small", "medium", "large", "melody"]

  def set_model(new_model="", seconds=30):
      global current_model  # Declare current_model as a global variable
      global model
      try:
          current_model
      except NameError:

          current_model = "init"
          print("Initializing Model")
      else:
          print("Current Model ", current_model)

      if new_model != current_model:

          model = musicgen.MusicGen.get_pretrained('large', device='cuda')
          model.set_generation_params(duration=seconds)
          current_model = new_model
          print("Loading", model, "Model...")

  already_imported = True
  print("Installed and Imported Neccessary Libraries")

#@markdown **Output Directory** *
directory = "/content/gdrive/My Drive/MusicGen" #@param {type:"string"}

#Check the model and set if its not already.
set_model(new_model=selected_model)

#@markdown *This is where we are saving the generated files.*

#@markdown *The default is set to save in your Google Drive in a folder called MusicGen.*

#@markdown **Limitation**: *Google Colab cannot save directly to your hard drive... this is why we use Google Drive.*

#@markdown *I dont recommend changing this unless you know what you are doing.

#@markdown **Generate Audio**
#@markdown Press Shift-Enter or press run on this cell.

#This generates a clean filepath if the directory doesnt exist
sanitized_prompt = ''.join(c for c in prompt if c.isalnum() or c.isspace())[:64]
directory_path = f'{directory}/{sanitized_prompt}'
os.makedirs(directory_path, exist_ok=True)

#This is where we generate the music
#Each Prompt gets its own clean directory (up to 64 characters)
#Inside the folder it will generate a .wav * the batch_size
#It will also make an .txt file with the unaltered prompt in the same directory
#This process is repeated for each prompt.


for i in range(batch_size):
    res = model.generate([prompt], progress=True)

    filename = f'{directory_path}/{i}.wav'
    audio_data = np.int16(res.cpu().numpy() / np.max(np.abs(res.cpu().numpy())) * 32767)
    audio_data = audio_data.reshape(-1, 1)  # Reshape to 2D array with one column
    write(filename, 32000, audio_data)
    # Save the full prompt in a text file
    with open(f'{directory_path}/prompt.txt', 'w') as f:
        f.write(prompt)
    del res

Already Installed. Skipping Install and Imports.
Current Model  large


# Out of Memory?

*   *If it runs out of memory try lowering some of the settings and then click Runtime at the top -> Restart Runtime*

# Bugs?
*   *Report Issues, Bugs and Suggestions on the* [Discord Server](https://discord.gg/nBbf99GR3d)

# Resources

*   ~Prompt Examples -- COMING SOON~

*   ~Prompt Guide - COMING SOON~

*   This script was created by Nick @ AudioIV [LinkTree](https://linktr.ee/blueyenick)


*   Discord Server - [Click Here](https://discord.gg/nBbf99GR3d)

# Disclaimer

*   This is provided for demonstration and educational purposes only. Please be aware that I am not accountable for the ways in which you employ this code or what you do with the results. It's your responsibility to use it wisely and ethically.

# TODO:

*   add knobs for other parameters (top_k, top_p, temperature...)
*   add input for melody model
*   add generate_continuation function for proper melody generation
*   add outpainting to extend tracks to longer than 30 seconds (this may end up being a seperate project)

# License

*   My Code license is MIT... but you need to research each imported library for its license before using this in any application.

# Credits

This notebook was created by [BluEyeNick](https://github.com/BluEyeNick) using the following open source libraries:

- [Google Colab](https://colab.research.google.com/) - For the notebook environment.

- [MusicGen](https://github.com/facebookresearch/audiocraft) by Facebook AI Research - For AI music generation.

- [PyTorch](https://pytorch.org/) - For deep learning model implementation.

- [NumPy](https://numpy.org/) - For numerical processing.

- [SciPy](https://www.scipy.org/) - For audio file handling.

The pre-trained MusicGen models used in this notebook were created and open sourced by researchers at Facebook AI Research.

This notebook is intended for educational and demonstration purposes only. Please review licenses and terms for any included libraries prior to commercial use.

Special thanks to the researchers and developers who created the amazing open source tools that made this notebook possible!